# Image Classification

In this notebook we train a convolutional neural network to classify images as either fire or koala. The notebook is structured as follows:

1. First we build a scraper to extract pictures of koalas and bush-fires and some other random noise.
2. We build a convolutional neural network

In [25]:
#Necessary Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
import tweepy
import pickle 
import time
import matplotlib.pyplot as plt
import requests as rq
from bing_image_downloader import downloader


## Bing Image Scraper

In [8]:
queries = ["koala", "bush fire"]

In [ ]:
for query in queries:
    downloader.download(query_string, limit=100,  
                        output_dir='image_dataset', 
                        adult_filter_off=False, 
                        force_replace=False, 
                        timeout=60)

## Get Twitter Images

In [26]:
image_tweets = pd.read_csv("data/image_tweets", index_col=0)
#Create list of image urls
image_urls = image_tweets["image_url"].unique()

In [28]:
#image_urls

In [30]:
image_urls.shape

(725,)